In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr
import pytesseract
from PIL import Image
import re
import json

def parse_document(image_path):
    # Open the image using PIL
    image = Image.open(image_path)

    # Use pytesseract to extract text
    text = pytesseract.image_to_string(image)

    # Define regular expressions to extract information
    data = {
        "filing_id": re.search(r"Filing ID\s*:\s*(\d+)", text),
        "document_id": re.search(r"Document ID\s*:\s*(\d+)", text),
        "bank_name": "FIRST COMMUNITY BANK" if "FIRST COMMUNITY BANK" in text else None,
        "debtor": {
            "name": re.search(r"1b\. INDIVIDUAL'S SURNAME FIRST PERSONAL NAME ADDITIONAL NAME\(S\)/INITIAL\(S\) SUFFIX\s+(.+?)\n", text),
            "address": re.search(r"1c\. MAILING ADDRESS\s+(.+?)\s+(LEAD HILL AR\s+\d+ USA)", text)
        },
        "secured_party": {
            "name": re.search(r"3a\. ORGANIZATION'S NAME\s+(.+?)\n", text),
            "address": re.search(r"3c\. MAILING ADDRESS:\s+(.+?)\s+(.+?)\s+(\d+ USA)", text)
        },
        "collateral": re.search(r"4\. COLLATERAL:\s+(.+?)\n\n", text, re.DOTALL)
    }

    # Extracted values need to be stripped and checked for None
    for key, value in data.items():
        if isinstance(value, re.Match):
            data[key] = value.group(1).strip()
        elif isinstance(value, dict):
            for sub_key, sub_value in value.items():
                if isinstance(sub_value, re.Match):
                    value[sub_key] = sub_value.group(1).strip()
                else:
                    value[sub_key] = None
        else:
            data[key] = None

    return data

# Path to the image file
image_path = '/content/4000028037756.tiff'

# Parse the document and print the result
parsed_data = parse_document(image_path)
print(json.dumps(parsed_data, indent=4))


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
{
    "filing_id": "4000028037756",
    "document_id": "17503743001",
    "bank_name": null,
    "debtor": {
        "name": "HUEBNER RICHIE HICKS",
        "address": null
    },
    "secured_party": {
        "name": "FIRST COMMUNITY BANK",
        "address": "CITY STATE |POSTAL CODE COUNTRY"
    },
    "collateral": "This financing statement covers the following collateral:"
}
